In [ ]:
# from google.colab import files
# files.upload() 
import pandas as pd
import glob
import os
!pip install --upgrade pycantonese
import pycantonese as pc
from pycantonese.word_segmentation import Segmenter

all_txt = glob.glob(os.getcwd() + "/*.txt")
if len(all_txt) > 0:
  corpus = []
  for txt in all_txt:
    f = open(txt, 'r')
    corpus.extend([x for x in f.read().strip().splitlines() if x])
    f.close()
  corpus_yue = corpus[::2]
  corpus_eng = corpus[1::2]
  df_txt = pd.DataFrame({'zh': corpus_yue, 'en': corpus_eng})
  # df_txt.to_csv('Netflix.csv', sep='\t', encoding='utf-8')

Requirement already up-to-date: pycantonese in /usr/local/lib/python3.6/dist-packages (3.0.0)


In [ ]:
all_csv = glob.glob(os.getcwd() + "/*.csv")  
 
df_from_each_file = (pd.read_csv(f,sep='\t',encoding='utf-8', index_col=0) for f in all_csv)
df = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
# Unify the character to traditional
!wget -qnc https://raw.githubusercontent.com/skydark/nstools/master/zhtools/zh_wiki.py https://raw.githubusercontent.com/skydark/nstools/master/zhtools/langconv.py
from langconv import *
for file in glob.glob(os.getcwd() + "/*.py"):
  os.remove(file)
def ConvTrad(x):
  # !pip install opencc
  # import opencc
  # converter = opencc.OpenCC('s2hk.json')
  return Converter('zh-hant').convert(x)
 
# segment Cantonese sentences
segmenter = Segmenter(
    allow = {'甘乃迪', '坦帕', '塞拉里昂', '唔淨至', '宗教', '儀式', '上漲', '功夫', '卑牌', '關註', "系咪", '就系', '我地', '你地', '佢地', '距地', '呢地', '呢到', '帶嚟'},
    disallow = {'系呢', '係呢', "喺呢", '呢個係'}
)
def CantoSeg(x):
  return ' '.join(pc.segment(x, cls=segmenter))
 
# replace unuified words to HK version and unify variants into single form
def haai5haai2Conv(x):
  if all(h in x.split(' ') for h in ['系', '係']):
    x = ' '.join(['喺' if len(x) == 1 and x == '係' else x for x in x.split(' ')])
    x = ' '.join(['係' if len(x) == 1 and x == '系' else x for x in x.split(' ')])
  return x

singleReplaceSeq = {
    "系": "係", 
    "里": "裏", 
    '距': '佢',
    '黎': '呢', 
    '果': '嗰', 
    'D': '啲',
    '卑': '俾'
}
def singleReplace(x):
  return ' '.join([singleReplaceSeq[x] if len(x) == 1 and x in singleReplaceSeq.keys() else x for x in x.split(' ')])

combo = [f + ' ' + s 
         for f in ['呢', '嗰', '一'] 
         for s in ['陣', '啲', '個', '輯', '份', '間', '地']]
def combineSequence(x):
  for c in combo:
    if c in x:
      x = x.replace(c, c[::2])
  return x


Replacement = {
    '噶': '嘅',
    '裡': '裏',
    '哩': '呢',
    '咯': '囉',
    '佐': '咗',
    '姖': '佢',
    '系咪': '係咪',
    '就系': '就係',
    '即系': '即係',
    '我地': '我哋',
    '你地': '你哋',
    '距地': '佢哋',
    '呢地': '呢啲',
    '嗰地': '嗰啲',
    '呢到': '呢度',
    '卑牌': '啤牌',
    '關註': '關注',
    '唔淨至': '唔淨止'
  }

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.rename(columns={'zh': 'yue', 'en': 'eng'})

df['yue'] = df['yue'].apply(lambda x: ConvTrad(x))
df['yue'] = df['yue'].apply(lambda x: CantoSeg(x))
df['yue'] = df['yue'].apply(lambda x: haai5haai2Conv(x))
df['yue'] = df['yue'].apply(lambda x: singleReplace(x))
df['yue'] = df['yue'].apply(lambda x: combineSequence(x))
df['yue'] = df['yue'].replace(Replacement, regex=True)

# Remove all punctuation and extra space
for column in df:
  df[column] = df[column].str.replace(r'[^\w\s]+', ' ')
  df[column] = df[column].str.replace(r' +', ' ')
  df[column] = df[column].str.strip()
 
# Let the tokenizer perform segment for Cantonese
# df['yue'] = df['yue'].str.replace(r' ', '')
df.dropna()
df = df.sample(frac=1).reset_index(drop=True)

print(f'Data size: {len(df)}')
df.head()

# from pprint import *
# corpus = pc.hkcancor()
# pprint(corpus.search(character='難用'))
# df[df['yue'].str.contains(' ')]

Data size: 9812


,yue,eng
0,唔覺 得 好 可笑 咩 我哋 製造 出 時速 可以 達到 130 英里 嘅 車 然後 以 1...,Isn t it absurd that we created cars that can ...
1,因為 佢地 要 瞭解 我 嘅 行動,because they want to know about my business
2,聚合 在 一齊 之外 無 其他 用途,like the nerves
3,三 個 細路 就係 o 係 呢度 俾 人 抓 左,And that s where the three kids got arrested
4,呢度 係 另一個 例子,So here s another example


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
train, test

(                                                    yue                                                eng
 551   唔準 掂 同 食 醃菜 唔準 坐 梳化 或者 屋企人 嘅 床 上 每次 經期 完 咗 要 洗...  I was not allowed to touch or eat pickles I wa...
 902                                        佢地 扮 怪 面 嚇 人                       They were making scary faces
 624                       唔會 搞到 我哋 變得 邪惡 女人型 變成 自由黨 人 咩                turn us into godless sissy liberals
 4636                              呢個 模式 可以 清晰 令 您 瞭解 佢哋         So it spells those out in very clean terms
 4635                                          幾多 萬 億 掌聲                        How many trillions Applause
 ...                                                 ...                                                ...
 5549                                  此 事件 因此 亦 被 不斷 調查                  and the investigation is going on
 9422                            我 過去 係 素食主義 者 唔 飲酒 唔食 煙      I was a vegetarian I was sober I didn t smoke
 6290  所以 我 開始 諗 如果 每個人 都 好似

In [ ]:
train.to_csv('train.csv', sep='\t', index=False, encoding='utf-8')
test.to_csv('test.csv', sep='\t', index=False, encoding='utf-8')